In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import numpy as np
import os
import time

In [ ]:
from pydrake.all import (
    StartMeshcat,
    CollisionCheckerParams,
    RobotDiagramBuilder,
    MeshcatVisualizer,
    Parser,
    LoadModelDirectives,
    ProcessModelDirectives,
    SceneGraphCollisionChecker,
    AutoDiffXd,
    RigidTransform_,
    IrisNp2Options,
    IrisParameterizationFunction,
    MathematicalProgram,
    HPolyhedron,
    IrisNp2,
    Hyperellipsoid,
    RandomGenerator,
)

In [ ]:
import src.iiwa_analytic_ik as iiwa_analytic_ik
import src.common as common

# Parameters

In [ ]:
directives_file = os.path.join(common.RepoDir(), "models/directives.dmd.yaml")
grasp_distance = 0.6
GC2 = -1
GC4 = -1
GC6 = -1
q_tilde_start = np.array([2.7165428034875583, -1.6020894107055366, -2.2310925417095118, -1.1273910985350377, -2.4662887005324756, -1.0705505639656554, -2.0129979825706337, -1.38])

# Set Up Environment

In [ ]:
meshcat = StartMeshcat()

In [ ]:
params = CollisionCheckerParams()
builder = RobotDiagramBuilder(time_step=0.0)
MeshcatVisualizer.AddToBuilder(builder.builder(), builder.scene_graph(), meshcat)
plant = builder.plant()

parser = Parser(plant)
package_xml_path = os.path.join(common.RepoDir(), "package.xml")
parser.package_map().AddPackageXml(package_xml_path)
directives = LoadModelDirectives(directives_file)
ProcessModelDirectives(directives, parser)

params.robot_model_instances = [
    plant.GetModelInstanceByName("iiwa_left"),
    plant.GetModelInstanceByName("iiwa_right")
]

plant.Finalize()
diagram = builder.Build()

params.model = diagram
params.edge_step_size = 0.01
checker = SceneGraphCollisionChecker(params)

In [ ]:
context = diagram.CreateDefaultContext()
diagram.ForcedPublish(context)

# Build Regions

## Set Up the Parametrization

In [ ]:
analytic_ik = iiwa_analytic_ik.Analytic_IK_7DoF(iiwa_analytic_ik.iiwa_alpha,
                                                iiwa_analytic_ik.iiwa_d,
                                                iiwa_analytic_ik.iiwa_limits_lower,
                                                iiwa_analytic_ik.iiwa_limits_upper)

def q_to_ee_target(q):
    global grasp_distance

    ad = isinstance(q[0], AutoDiffXd)
    T = AutoDiffXd if ad else float
    
    tf_goal = analytic_ik.FK(q)
    ang = (180 - 2. * 68.) * np.pi / 180.
    c, s = np.cos(ang), np.sin(ang)
    tf_goal[:-1,:-1] = tf_goal[:-1,:-1] @ np.array([[-1, 0, 0], [0, 1, 0], [0, 0, -1]]) @ np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])
    tf_goal[:-1,-1] = tf_goal[:-1,-1] + tf_goal[:-1,:-1] @ np.array([0, 0, -grasp_distance])
    tf_goal[:-1,-1] = tf_goal[:-1,-1] + np.array([0, -0.765, 0])
    return RigidTransform_[T](tf_goal)

def parameterization(q_tilde):
    global GC2, GC4, GC6
    q_full = np.zeros(14, dtype=type(q_tilde[0]))
    q_full[:7] = q_tilde[:7]
    tf_goal = q_to_ee_target(q_tilde[:7])
    psi = q_tilde[7]
    q_follower = analytic_ik.IK(tf_goal, [GC2, GC4, GC6], psi)
    q_full[7:] = q_follower
    return q_full

In [ ]:
plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, parameterization(q_tilde_start))
diagram.ForcedPublish(context)

In [ ]:
iris_options = IrisNp2Options()
iris_options.parameterization = IrisParameterizationFunction(parameterization, 8)

iris_options.sampled_iris_options.verbose = True

iris_prog = MathematicalProgram()
iris_options.sampled_iris_options.prog_with_additional_constraints = iris_prog
q_tilde_vars = iris_prog.NewContinuousVariables(8, "q_tilde")

## Set Up the Additional Constraints

In [ ]:
# This constraint enforces reachability.
def unclipped_vals(q_tilde):
    global GC2, GC4, GC6
    tf_goal = q_to_ee_target(q_tilde[:7])
    psi = q_tilde[7]
    unclipped_vals = analytic_ik.IK(tf_goal, [GC2, GC4, GC6], psi, return_unclipped_vals=True)
    return unclipped_vals

iris_prog.AddConstraint(unclipped_vals,
                        -np.ones(4),
                        np.ones(4),
                        q_tilde_vars)

In [ ]:
# This constraint enforces the subordinate arm's joint limits.
def subordinate_arm_config(q_tilde):
    global GC2, GC4, GC6
    tf_goal = q_to_ee_target(q_tilde[:7])
    psi = q_tilde[7]
    q_follower = analytic_ik.IK(tf_goal, [GC2, GC4, GC6], psi)
    return q_follower

iris_prog.AddConstraint(subordinate_arm_config,
                        iiwa_analytic_ik.iiwa_limits_lower,
                        iiwa_analytic_ik.iiwa_limits_upper,
                        q_tilde_vars)

## Grow Regions for Hand-Chosen Seed Points

In [ ]:
# The self-motion parameter is circle-valued (wraps around at 2pi), but we ignore that for now.
domain_lower = np.hstack((iiwa_analytic_ik.iiwa_limits_lower, [-2.0 * np.pi]))
domain_upper = np.hstack((iiwa_analytic_ik.iiwa_limits_upper, [ 2.0 * np.pi]))
domain = HPolyhedron.MakeBox(domain_lower, domain_upper)

In [ ]:
region = IrisNp2(checker, Hyperellipsoid.MakeHypersphere(1e-2, q_tilde_start), domain, iris_options)
print("Number of faces", len(region.b()))

## Visualize a Walk around the Region

In [ ]:
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

q_tilde = q_tilde_start.copy()
rng = RandomGenerator()
while True:
    q_tilde_next = region.UniformSample(rng)
    for t in np.linspace(0, 1, 50):
        q = parameterization(t * q_tilde_next  + (1 - t) * q_tilde)
        plant.SetPositions(plant_context, q)
        diagram.ForcedPublish(context)
        time.sleep(0.02)
    q_tilde = q_tilde_next.copy()

# Plan with GcsTrajectoryOptimization

## Actually Do the Planning

## Remap and Retime Trajectory

## Visualize and Record the Plan